# Prepare flight delay data

To start, let's import the Python libraries and modules we will use in this notebook.

In [0]:
import datetime
from pyspark.sql.types import *
from pyspark.sql.functions import unix_timestamp
import math
from pyspark.sql import functions as F

First, let's execute the below command to make sure all three tables were created.
You should see an output like the following:

| database | tableName | isTemporary |
| --- | --- | --- |
| default | airport_code_loca... | false |
| default | flight_delays_wit... | false |
| default | flight_weather_wi... | false |

In [0]:
spark.sql("show tables").show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| default|airport_code_loca...|      false|
| default|flight_delays_wit...|      false|
| default|flight_weather_wi...|      false|
|        |  flight_delays_view|       true|
+--------+--------------------+-----------+



Now execute a SQL query using the `%sql` magic to select all columns from flight_delays_with_airport_codes. By default, only the first 1,000 rows will be returned.

In [0]:
%sql
select * from flight_delays_with_airport_codes

Year,Month,DayofMonth,DayOfWeek,Carrier,CRSDepTime,DepDelay,DepDel15,CRSArrTime,ArrDelay,ArrDel15,Cancelled,OriginAirportCode,OriginAirportName,OriginLatitude,OriginLongitude,DestAirportCode,DestAirportName,DestLatitude,DestLongitude
2013,4,19,5,DL,837,-3,0,1138,1,0,0,DTW,Detroit Metro Wayne County,42.2125,-83.35333333,MIA,Miami International,25.79527778,-80.29
2013,4,19,5,DL,1705,0,0,2336,-8,0,0,SLC,Salt Lake City International,40.78833333,-111.9777778,JFK,John F. Kennedy International,40.64,-73.77861111
2013,4,19,5,DL,600,-4,0,851,-15,0,0,PDX,Portland International,45.58861111,-122.5969444,SLC,Salt Lake City International,40.78833333,-111.9777778
2013,4,19,5,DL,1630,28,1,1903,24,1,0,STL,Lambert-St. Louis International,38.74861111,-90.37,DTW,Detroit Metro Wayne County,42.2125,-83.35333333
2013,4,19,5,DL,1615,-6,0,1805,-11,0,0,CVG,Cincinnati/Northern Kentucky International,39.04888889,-84.66777778,LAX,Los Angeles International,33.9425,-118.4080556
2013,4,19,5,DL,1726,-1,0,1818,-19,0,0,ATL,Hartsfield-Jackson Atlanta International,33.63666667,-84.42777778,STL,Lambert-St. Louis International,38.74861111,-90.37
2013,4,19,5,DL,1900,0,0,2133,-1,0,0,STL,Lambert-St. Louis International,38.74861111,-90.37,ATL,Hartsfield-Jackson Atlanta International,33.63666667,-84.42777778
2013,4,19,5,DL,2145,15,1,2356,24,1,0,ATL,Hartsfield-Jackson Atlanta International,33.63666667,-84.42777778,SLC,Salt Lake City International,40.78833333,-111.9777778
2013,4,19,5,DL,2157,33,1,2333,34,1,0,ATL,Hartsfield-Jackson Atlanta International,33.63666667,-84.42777778,AUS,Austin - Bergstrom International,30.19444444,-97.67
2013,4,19,5,DL,1900,323,1,2055,322,1,0,DCA,Ronald Reagan Washington National,38.85138889,-77.03777778,ATL,Hartsfield-Jackson Atlanta International,33.63666667,-84.42777778


Now let's see how many rows there are in the dataset.

In [0]:
%sql
select count(*) from flight_delays_with_airport_codes

count(1)
2719418


Based on the `count` result, you can see that the dataset has a total of 2,719,418 rows (also referred to as examples in Machine Learning literature). Looking at the table output from the previous query, you can see that the dataset contains 20 columns (also referred to as features).

Because all 20 columns are displayed, you can scroll the grid horizontally. Scroll until you see the **DepDel15** column. This column displays a 1 when the flight was delayed at least 15 minutes and 0 if there was no such delay. In the model you will construct, you will try to predict the value of this column for future data.

Let's execute another query that shows us how many rows do not have a value in the DepDel15 column.

In [0]:
%sql
select count(*) from flight_delays_with_airport_codes where DepDel15 is null

count(1)
27444


Notice that the `count` result is 27444. This means that 27,444 rows do not have a value in this column. Since this value is very important to our model, we will need to eliminate any rows that do not have a value for this column.

Next, scroll over to the **CRSDepTime** column within the table view above. Our model will approximate departure times to the nearest hour, but departure time is captured as an integer. For example, 8:37 am is captured as 837. Therefore, we will need to process the CRSDepTime column, and round it down to the nearest hour. To perform this rounding will require two steps, first you will need to divide the value by 100 (so that 837 becomes 8.37). Second, you will round this value down to the nearest hour (so that 8.37 becomes 8).

Finally, we do not need all 20 columns present in the flight_delays_with_airport_codes dataset, so we will pare down the columns, or features, in the dataset to the 12 we do need.

Using `%sql` magic allows us view and visualize the data, but for working with the data in our tables, we want to take advantage of the rich optimizations provided by DataFrames. Let's execute the same query using Spark SQL, this time saving the query to a DataFrame.

In [0]:
dfFlightDelays = spark.sql("select * from flight_delays_with_airport_codes")

Let's print the schema for the DataFrame.

In [0]:
print(dfFlightDelays.dtypes)

[('Year', 'int'), ('Month', 'int'), ('DayofMonth', 'int'), ('DayOfWeek', 'int'), ('Carrier', 'string'), ('CRSDepTime', 'int'), ('DepDelay', 'int'), ('DepDel15', 'int'), ('CRSArrTime', 'int'), ('ArrDelay', 'int'), ('ArrDel15', 'int'), ('Cancelled', 'int'), ('OriginAirportCode', 'string'), ('OriginAirportName', 'string'), ('OriginLatitude', 'double'), ('OriginLongitude', 'double'), ('DestAirportCode', 'string'), ('DestAirportName', 'string'), ('DestLatitude', 'double'), ('DestLongitude', 'double')]


Notice that the DepDel15 and CRSDepTime columns are both `string` data types. Both of these features need to be numeric, according to their descriptions above. We will cast these columns to their required data types next.

## Perform data munging

To perform our data munging, we have multiple options, but in this case, we’ve chosen to take advantage of some useful features of R to perform the following tasks:

* Remove rows with missing values
* Generate a new column, named “CRSDepHour,” which contains the rounded down value from CRSDepTime
* Pare down columns to only those needed for our model

In [0]:
# Select only the columns we need, casting CRSDepTime as long and DepDel15 as int, into a new DataFrame
dfflights = spark.sql("SELECT OriginAirportCode, OriginLatitude, OriginLongitude, Month, DayofMonth, cast(CRSDepTime as long) CRSDepTime, DayOfWeek, Carrier, DestAirportCode, DestLatitude, DestLongitude, cast(DepDel15 as int) DepDel15 from flight_delays_with_airport_codes")

# Delete rows containing missing values
dfflights = dfflights.na.drop("any")

# Round departure times down to the nearest hour, and export the result as a new column named "CRSDepHour"
dfflights = dfflights.withColumn("CRSDepHour", F.floor(F.col('CRSDepTime') / 100))

display(dfflights)

OriginAirportCode,OriginLatitude,OriginLongitude,Month,DayofMonth,CRSDepTime,DayOfWeek,Carrier,DestAirportCode,DestLatitude,DestLongitude,DepDel15,CRSDepHour
SAT,29.53388889,-98.46916667,5,1,1350,3,MQ,ORD,41.97944444,-87.9075,0,13
SAT,29.53388889,-98.46916667,5,2,1350,4,MQ,ORD,41.97944444,-87.9075,0,13
SAT,29.53388889,-98.46916667,5,3,1350,5,MQ,ORD,41.97944444,-87.9075,1,13
SAT,29.53388889,-98.46916667,5,4,1350,6,MQ,ORD,41.97944444,-87.9075,0,13
SAT,29.53388889,-98.46916667,5,5,1350,7,MQ,ORD,41.97944444,-87.9075,0,13
SAT,29.53388889,-98.46916667,5,6,1350,1,MQ,ORD,41.97944444,-87.9075,0,13
SAT,29.53388889,-98.46916667,5,7,1350,2,MQ,ORD,41.97944444,-87.9075,0,13
SAT,29.53388889,-98.46916667,5,8,1350,3,MQ,ORD,41.97944444,-87.9075,1,13
SAT,29.53388889,-98.46916667,5,9,1350,4,MQ,ORD,41.97944444,-87.9075,0,13
SAT,29.53388889,-98.46916667,5,10,1350,5,MQ,ORD,41.97944444,-87.9075,1,13


In [0]:
# Create a Temporary Table / View with clean data from the DataFrame 
dfflights.createOrReplaceTempView("flight_delays_view")

Now let's take a look at the resulting data. Take note of the **CRSDepHour** column that we created, as well as the number of columns we now have (12). Verify that the new CRSDepHour column contains the rounded hour values from our CRSDepTime column.

In [0]:
%sql
select * from flight_delays_view

OriginAirportCode,OriginLatitude,OriginLongitude,Month,DayofMonth,CRSDepTime,DayOfWeek,Carrier,DestAirportCode,DestLatitude,DestLongitude,DepDel15,CRSDepHour
SAT,29.53388889,-98.46916667,5,1,1350,3,MQ,ORD,41.97944444,-87.9075,0,13
SAT,29.53388889,-98.46916667,5,2,1350,4,MQ,ORD,41.97944444,-87.9075,0,13
SAT,29.53388889,-98.46916667,5,3,1350,5,MQ,ORD,41.97944444,-87.9075,1,13
SAT,29.53388889,-98.46916667,5,4,1350,6,MQ,ORD,41.97944444,-87.9075,0,13
SAT,29.53388889,-98.46916667,5,5,1350,7,MQ,ORD,41.97944444,-87.9075,0,13
SAT,29.53388889,-98.46916667,5,6,1350,1,MQ,ORD,41.97944444,-87.9075,0,13
SAT,29.53388889,-98.46916667,5,7,1350,2,MQ,ORD,41.97944444,-87.9075,0,13
SAT,29.53388889,-98.46916667,5,8,1350,3,MQ,ORD,41.97944444,-87.9075,1,13
SAT,29.53388889,-98.46916667,5,9,1350,4,MQ,ORD,41.97944444,-87.9075,0,13
SAT,29.53388889,-98.46916667,5,10,1350,5,MQ,ORD,41.97944444,-87.9075,1,13


Now verify that the rows with missing data for the **DepDel15** column have been removed.

In [0]:
%sql
select count(*) from flight_delays_view

count(1)
2691974


You should see a count of **2,691,974**. This is equal to the original 2,719,418 rows minus the 27,444 rows with missing data in the DepDel15 column.

Now save the contents of the temporary view into a new DataFrame.

In [0]:
dfFlightDelays_Clean = spark.sql("select * from flight_delays_view")

## Export the prepared data to persistent a global table

There are two types of tables in Databricks. 

* Global tables, which are accessible across all clusters
* Local tables, which are available only within one cluster

To create a global table, you use the `saveAsTable()` method. To create a local table, you would use either the `createOrReplaceTempView()` or `registerTempTable()` method.

The `flight_delays_view` table was created as a local table using `createOrReplaceTempView`, and is therefore temporary. Local tables are tied to the Spark/SparkSQL Context that was used to create their associated DataFrame. When you shut down the SparkSession that is associated with the cluster (such as shutting down the cluster) then local, temporary tables will disappear. If we want our cleansed data to remain permanently, we should create a global table. 

Run the following to copy the data from the source location into a global table named `flight_delays_clean`.

In [0]:
dfFlightDelays_Clean.write.mode("overwrite").saveAsTable("flight_delays_clean")

In [0]:
%sql
DESCRIBE DETAIL flight_delays_clean

format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion
delta,9f75723f-21a4-4e63-948d-337081d0992f,spark_catalog.default.flight_delays_clean,null,dbfs:/user/hive/warehouse/flight_delays_clean,2022-11-23T21:50:34.300+0000,2022-11-24T04:36:52.000+0000,List(),4,13707170,Map(),1,2


## Next step

Continue to the next notebook, [02 Weather Data Preparation]($./02%20Weather%20Data%20Preparation).